In [11]:
from core import RootCore
import os
import shutil
import hashlib
from PIL import Image
import numpy as np
import requests
import json

In [12]:
# 崇州口腔医院
# tenantid = '5d22c4f392a86bc0c39b059f'
# data_path = '../../wangbilin/chongzhou0715'
# result_path = '../../wangbilin/chongzhou0715Result'
# except_path = '../../wangbilin/chongzhou0715Except'
# # 峨眉山口腔医院
# tenantid = '5d22e07a92a86bc0c39b15cc'
# data_path = '../../wangbilin/emeishan'
# result_path = '../../wangbilin/emeishanResult'
# except_path = '../../wangbilin/emeishanExcept'

# 光华口腔
tenantid = '5d22c96092a86bc0c39b08ae'
data_path = '../../wangbilin/guanghua'
result_path = '../../wangbilin/guanghuaResult'
except_path = '../../wangbilin/guanghuaExcept'

filenames = os.listdir(data_path)
# print(filenames)
print('{}共有{}个根尖片'.format(data_path,len(filenames)))

../../wangbilin/guanghua共有17个根尖片


In [13]:
# 图片的md5
def imageMD5(path):
    fd = np.array(Image.open(path)) 
    fmd5 = hashlib.md5(fd)
    return fmd5.hexdigest()

# 上传图片方法
def uploadTask(file_path,mark_path,bbox):
    files = {
      "file" : open(file_path, "rb"),
      "markfile" : open(mark_path, "rb")
    }
    url = 'https://rootai.mooplab.com/mp/task'
    headers = {
        'Token':'Welcome1'
    }
    data = {
        'tenantid':tenantid,
        'bboxes':json.dumps(bbox),
        'md5':imageMD5(mark_path)
    }
    r = requests.post(url, data,headers=headers, files=files)
    print(r.json())
    return r.json()

In [14]:
# init
core = RootCore.RootCore(
    './data/detection_state_dict.pth',
    './data/score_state_dict.pth',
    '', # model encryption not implemented for now
    device_name='cpu'
)

In [15]:
result = []
count = 0
for file in filenames:
    count += 1
    if file == '.ipynb_checkpoints':
        continue
    file_path = '{}/{}'.format(data_path,file)
    bboxes = core.detect(file_path)
    score_bboxes = core.bbox_score(file_path, bboxes)
    print(count,file_path)
    if score_bboxes:
        item = {
            'file_path':file_path,
            'score_bboxes':score_bboxes,
            'file_name':file
        }
        result.append(item)
    else:
        shutil.copy(file_path,except_path+'/')
        print('{},未找到根管，已拷贝至：{}'.format(file_path,except_path))
    
    # 测试
#     if count == 10:
#         break
print(result)

1 ../../wangbilin/guanghua/Z1-6-27.jpg
2 ../../wangbilin/guanghua/H3-6-21.jpg
3 ../../wangbilin/guanghua/H7-6-8.jpg
4 ../../wangbilin/guanghua/W8-6-6.jpg
5 ../../wangbilin/guanghua/H5-6-19.jpg
6 ../../wangbilin/guanghua/W4-6-20.jpg
7 ../../wangbilin/guanghua/W7-6-12.jpg
8 ../../wangbilin/guanghua/H1-7-5.jpg
9 ../../wangbilin/guanghua/H4-6-20.jpg
10 ../../wangbilin/guanghua/H2-6-24.jpg
11 ../../wangbilin/guanghua/W3-6-25.jpg
12 ../../wangbilin/guanghua/H6-6-15.jpg
13 ../../wangbilin/guanghua/W2-7-9.jpg
14 ../../wangbilin/guanghua/W1-.jpg
15 ../../wangbilin/guanghua/W6-6-20.jpg
16 ../../wangbilin/guanghua/Z2-6-19.jpg
17 ../../wangbilin/guanghua/W5-6-20.jpg
[{'file_path': '../../wangbilin/guanghua/Z1-6-27.jpg', 'score_bboxes': [[663.18994140625, 27.170730590820312, 948.502685546875, 398.8157958984375, 82.81810760498047]], 'file_name': 'Z1-6-27.jpg'}, {'file_path': '../../wangbilin/guanghua/H3-6-21.jpg', 'score_bboxes': [[171.46133422851562, 306.4106750488281, 482.9976806640625, 868.522460

In [16]:
for r in result:
    for i, score_bbox in enumerate(r['score_bboxes']):
        result_file = core.draw_score_bboxes(
            r['file_path'],
            '{}/{}-'.format(result_path,i),
            [score_bbox],
            bbox_color_name='lime',
            draw_score=False
        )
        # 上传到服务器
        uploadTask(r['file_path'],result_file,score_bbox)
        print('bbox {}:\n{}\npath: {}\n'.format(i, score_bbox, result_file))

{'data': '操作成功', 'message': '', 'status': True}
bbox 0:
[663.18994140625, 27.170730590820312, 948.502685546875, 398.8157958984375, 82.81810760498047]
path: ../../wangbilin/guanghuaResult/0-Z1-6-27.jpg

{'data': '操作成功', 'message': '', 'status': True}
bbox 0:
[171.46133422851562, 306.4106750488281, 482.9976806640625, 868.5224609375, 86.27974700927734]
path: ../../wangbilin/guanghuaResult/0-H3-6-21.jpg

{'data': '操作成功', 'message': '', 'status': True}
bbox 0:
[384.7535705566406, 68.40559387207031, 815.8665771484375, 467.86151123046875, 79.92337036132812]
path: ../../wangbilin/guanghuaResult/0-H7-6-8.jpg

{'data': '操作成功', 'message': '', 'status': True}
bbox 0:
[308.13726806640625, 285.37725830078125, 717.162109375, 695.585693359375, 80.04814147949219]
path: ../../wangbilin/guanghuaResult/0-W8-6-6.jpg

{'data': '操作成功', 'message': '', 'status': True}
bbox 0:
[278.3778076171875, 171.21878051757812, 726.4722900390625, 601.9197387695312, 83.86164855957031]
path: ../../wangbilin/guanghuaResult/0-